ref: https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

In [1]:
import os
os.environ['KERAS_BACKEND']='theano'
#os.environ['THEANO_FLAGS']="floatX=float64,device=cpu"
os.environ['THEANO_FLAGS']="floatX=float32,device=cuda"

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
import keras.backend as K
import numpy as np
import random
import sys

Using Theano backend.
Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GTX 1080 (0000:01:00.0)


In [3]:
text = open("sdyxz_all.txt").read().lower().replace("\n\n", "\n").replace("\n\n", "\n")
print('corpus length:', len(text))

corpus length: 898641


In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 4127


In [5]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
maxlen = 30
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 898611


Vectorization

In [7]:
X = np.zeros((len(sentences), maxlen), dtype=np.int32)
y = np.zeros((len(sentences),), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t] = char_indices[char] 
    y[i] = char_indices[next_chars[i]]

The model

In [8]:
def sparse_top_k_categorical_accuracy(y_true, y_pred, k=5):
        return K.mean(K.in_top_k(y_pred, K.max(y_true, axis=-1), k))

In [9]:
model = Sequential()
model.add(Embedding(len(chars), 128))
model.add(LSTM(512, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, 
              metrics=['accuracy', sparse_top_k_categorical_accuracy])

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]        
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(400):
            x = np.zeros((1, maxlen))
            for t, char in enumerate(sentence):
                x[0, t] = char_indices[char]

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char                    
        print(generated)


--------------------------------------------------
Iteration 1
Epoch 1/1
898611/898611 [==============================] - 575s - loss: 5.0022 - acc: 0.2205 - sparse_top_k_categorical_accuracy: 0.3658   

----- diversity: 0.2
----- Generating with seed: "出力抵抗金兵的大將罷免的罷免，殺頭的殺頭。花花江山，雙手送將"
出力抵抗金兵的大將罷免的罷免，殺頭的殺頭。花花江山，雙手送將一個大小子，你這麼一個事，我們也不知道。」黃蓉道：「你不是你，你們的話，你不是你。」郭靖道：「你不是我的話，我也不知道。」黃蓉道：「你們一個人，你們一個個人，你們一個個人，你們一個個人。」郭靖道：「你不是你的。」郭靖道：「你不是我的。」郭靖道：「你不是我的話，你不是你的。」郭靖道：「你不是你的話，我不是你的。」郭靖道：「你不是我的話，你這麼一個事，我不是你的。」郭靖道：「我不是你的。」郭靖道：「你不是你的。」郭靖道：「你不是你的。」郭靖道：「你不是你的。」郭靖道：「你不是我的話，你不是你的。」郭靖道：「你不是我的話，你們說道：『你們這麼？」郭靖道：「你不是你，你們不見你。」黃蓉道：「你不是你的。」郭靖道：「你不是你的。」郭靖道：「你們一個，你們這麼一個事，你不是我的。」郭靖道：「你不是你的話，你不是你的什麼事，你這麼一個事，你是你的。」郭靖道：「我們不知道。」郭靖道：「你不是我的。」郭靖道

----- diversity: 0.5
----- Generating with seed: "出力抵抗金兵的大將罷免的罷免，殺頭的殺頭。花花江山，雙手送將"
出力抵抗金兵的大將罷免的罷免，殺頭的殺頭。花花江山，雙手送將兩人，你一個小子，怎麼？」黃蓉道：「你不是我的道：「你不是我的生子，我來不定要有。」黃蓉道：「你不是你說，心中你暗兒，你不是你要來。」郭靖道：「你瞧他們的。」郭靖道：「我不是。」郭靖道：「我不說，你們知道。」郭靖道：「你們下去。」黃蓉道：「你在這裡，我與你這位師父自然不明。」郭靖伸手在這裡，說道：「不可我，你們到這裡去，那麼你們也不是。

## 其他結果
Adam 
```python
maxlen=32
step = 3
model.add(Embedding(len(chars), 256))
model.add(LSTM(512, input_shape=(maxlen, len(chars))))
```
### 紅樓夢
291291/291291 [==============================] - 194s - loss: 0.9957 - acc: 0.7413 - sparse_top_k_categorical_accuracy: 0.9159 

----- diversity: 0.2
----- Generating with seed: "我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即"
我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即刻差什么事，不過明白几天，我們也不用找，我再告訴你．我說你們奶奶在園子里找去，大爺回去再回。”鳳姐睜著眼，說道：“好妹妹，你好生日罷．我听見說城里有年紀，大家議保，越發來了。”
　　一語未了，只見香菱進來忙打發人進來． 見了這些話，心里卻不自在，听如此說，便自己勉強說道：“那怕你們來攪，就來這兩日，我也不敢說出來． "一面說，一面向他說道：“你那里得得這么說，我就不信了． 獨有那些人送到園里，叫他在外頭逛去。”寶玉道：“我們還不知道，我也不敢去。”賈母笑道：“這倒好，只是你說的也不在家。”賈璉道：“你說他在這里，沒有吃兩個．我倒不好了，就只法子吃罷． "黛玉道：“這倒是他們的使你． "寶玉笑道：“你既有走過來，我就立手站著．我們再細吃，我們娘娘才死，不有事忘了他的去， 不敢惊動他． "平儿也顧著說道：“你要有什么話！你不這么說，我還要吵嚷，有什么難了，就說嘴儿！你是你的丫頭，我就打發平儿住

----- diversity: 0.5
----- Generating with seed: "我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即"
我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即轉過一個好些儿子：你死了！把昨日太太有了大娘的了，就要殺人恨這些事，那里還來使罷． "薛姨媽道：“我說你來，就跑出來浪人都躲在嬸子去． "鳳姐听了這話，又被他道：“你不要叫你，穿著你媳婦。”那丫頭听見鳳姐如何答言，只听屋里的無著正在那里呢！我們看著也是個吃知的。”林黛玉听了，笑道：“你們听听，說個什么？"翠縷道：“拿去罷．誰叫你送去呢？"寶釵笑道：“我還有一句話可使得呢？ "薛姨媽道：“你要怎么樣管家事，你怎么不怨？"襲人道：“怎么還得的？"寶玉道：“他怎么不傷心，我們不敢遲。”妙玉听了，又笑道：“這也是有的． "寶玉笑道：“我們不敢常使，只賞了來瞧你。”寶玉道：“你去罷，我還有几件東西不要來呢！"一面說，一面催他起來．
　　襲人見他說了，心下想道：“不是我一生有睜眼色止住了。”一面說，一面拉了兩個丫頭，卻一一急，又唬一個碧點頭儿說道：“天天有什么事情，來就是了。”紅玉道：“你還不知道呢，

----- diversity: 1.0
----- Generating with seed: "我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即"
我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即刻做些樣儿，說我和別人行正．他倒了一點儿就來了。”秦氏道：“可不是有急的呢．沒八個定感，正是官司一個人去，是他們三次的理我，上頭又傷起心來，你也喜歡，只一一步問一回．一個人道：既有個，主仆今又作出力經似的，如今這也就可以高處？"先生道"倒是`蟠記》出來叫作`誰周下寶玉原為是家時為出來帶去世襲人知道場！下一張先生的小人剛是天天古怪的東西，若我這几年，怎么就得了？我擱在窗戶一樣子了。”寶玉笑道：“你起這正經事皆想多說呢．世人對景！況且能別全無相親，世兄也遠，故曰"實錄重生人去更更好。”寶玉正說他有銀兩， 便用手拉著襲人等處來問是鴛鴦．黛玉便問"外頭說：“我也是個丫頭，也怨不得．你早死了几天，該叫我去睡覺去． "一面說，一面苦了．接他走到門前，黛玉回身看了，說：“有了罷，不然我卻好生怕，別的恩典，太太和你屋里的人，怨命而好，況且你在姑娘身上我摸，送個司馬姊妹們一個好老婆子送去， 到五十點了，就將

----- diversity: 1.2
----- Generating with seed: "我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即"
我的也是一般，他說姐姐就是說我．我那邊的人有怨我的，姐姐听見也即不傷心，所以如何使得呢？ "黛玉道：“我倒是出去．不因我天天那何等爽快呢？"黛玉道：“這么算呢！我要叫了他來收拾，他明儿打出來了么？"寶玉因知不覺心里不答言，問他和他歎道：“所以我那閻王寶玉站的是唬無聲個戲与寶玉還好，眾人既辛金姓，各祖，又叫小儿，說話些世事．看到他們，更信，說閒閒話，就用不留這些東西．宁可服坏了法年， 讓他好姐妹妹并可与迎了牆去有罪． 我又沒口應了几個下去，做的理子，作一個破題，再作這些百包金星．前日听見可怜我的，不知道我的心事，就只好好的又上了本事，就是他們姊妹們相心．其中父親弟父索知道，必要深為听听，即來得身上認真辦些須去好方的，故自越發起心要緊的人方來．看起黛玉住至眾人，賈璉素日大家周小心事， 不免又勸鳳姐想道：“你先替二爺找罷，我勸几天假心時，昨日老太太气坏了，二爺就要請出這事來。”薛蟠道：“你們都窮罷．誰叫他男人的， 各自找來！我在老太太跟前討老奶奶，老爺們家和

### 三國演義
----- diversity: 0.2
----- Generating with seed: "与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報"
与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報國賊！非等閒之謀，不可不樂。”董云大怒曰：“汝且如平气者，必盡追之。”黃忠曰：“某愿當此天下？”孔明曰：“司馬懿非二人，欲使大將不可，主公可自取其禍。若得同而行之，令兄弟夏侯楙征諸葛亮之德也。”誕大怒，欲斬田修。使者至邊下，早有細作在帳中坐，分了人傷，人皆不信，死。眾官又有异心。孔明隨后引兵殺奔冀州。兩軍相謂曰：“吾能用奇胜之人，人皆知之，早知汝勿擒矣。”昭大怒曰：“汝尚欲降，何以知賊！”
　　于是吳兵內隊兵五百，四面圍合，前隊四散。急急退兵，直撞入時，張飛回馬，大叫一聲而立。張英棄全歎曰：“吾看關隘口，我等皆必來取汝南郡。”一面壯士盡數千人。時天色已失，人有人而走；只得睡時，孔明親自至許都，怏拔不敢來。玄德大惊，坐于帳前，囑曰：“周瑜、關、張兄弟、妻子蘭、董承之禍。”關公曰：“長子熙出，未曾有德；尊以王命忘罪流見，教秦蓋同吳王平，到彼處而見財物矣。”先主大怒，命將來拜謝。懿分付龐統降魏主

----- diversity: 0.5
----- Generating with seed: "与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報"
与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報至廟！”曹休大惊。昭又告孫權：“晉兵去間，臣未及所乘。”操然其言，遂命孫乾先往江夏求救，望為內宮。眾官勸免。郭嘉曰：“都督知江夏諸葛亮之計矣。”云長曰：“此因乃為上關之意；不必疑來；夫人是勸主公。”孔明曰：“子敬見公相見。”操曰：“將在其道而用徐氏之也。臣未能成，反遭他心；更兼曹操以仁德待之，諸葛亮亦可問也。”瑜曰：“吾有問先生報仇！今日便來管文台下以示之師，不殺汝也。汝父兄不可惜身，以蟬在大，死豈甘露之面，則世間有烏妙；吾非國家效力，正欲侵魏，而乃魏國之心，其勢危急，非曹睿之說詞所見，何必問？”昭曰：“吾有奉詔，明無异心。”遂仍將入拜。懿曰：“君臣托重荐之人，也不至地。今幸將軍深信，恨無面弓，如推不知！”權以書相覷，使者不可。紹曰：“彼專怀方得周瑜、張魯相見，必生异憑。”就馬急救。孔明問取安之狀，望同入。茶畢，孔明曰：“汝事急矣。布雖疑而回吳，欲殺吳公，豈肯辱乎？”玄德不從，來問從事。懿

----- diversity: 1.0
----- Generating with seed: "与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報"
与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報國賊！非等閒之謀，不可与用。”璋曰：“倘魏兵先至，休受死眾。”三軍盡皆入去。懿自引三万蠻兵，去守葭萌關。前日車馬引大軍江岸，兩隊馬人圍住周來沖突，追殺至關下，奪得孫權”權觀之，乃魏延中了一枝軍馬。衛曰：“前面魏將殺十余道，何不作二千人？”瑜曰：“司馬師以兵廢之，必將全家軍馬祭降，免生叛質，愿丞相諸葛亮之說，兄弟吾回決矣。將軍可再來，万何難保？吾今放上而去，投明白攻而進，發兵出戰，斷其歸路。吾今大軍万大，望風攻吾，此號疑矣。”睿欣然其言。忽一人問曰：“將軍何故至此？”延曰：“夫人勿惊。”飛曰：“已要半途，遣兵到矣。”卓問：“誰可為使？”孔明曰：“三千余字，若未知兵法可也。”云長曰：“今年十五歲矣，當世之以為我之心也。今某當先伐之，以安天下！”成都曰：“劉備盡如天下大小，真親力未立。若勢孤窮于長沙，死敢成矣。又不疑使董卓，孤將取西川也。”懿曰：“蜀兵千余万，望羌兵大至矣！”睿曰：“有何机事？”

----- diversity: 1.2
----- Generating with seed: "与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報"
与伏完。量汝忠義，必不負朕。”順泣曰：“臣感陛下大恩，敢不以死報至廟！吾乃國家舊有殺妻；若吾大將可重賞，祖尸面指，指吾可推：張、黃、南、北乎？”懿曰：“可請丞相放去問之，某為分兩路。”孔明曰：“西北人之言相至，未嘗少道：使德操出，待其羽典，要引兵至此。”呂布受了要回，賞勞過來。古堂興拜書飲宴，先主泣謂曰：“卿言常在。”
　　吉平辭了陳宮，敕令瑜回南鄭，先望蜀錦木成，砍与羌兵皆解。未及交鋒，吳兵請回許昌。吳主孫亮托糧食來路，迎入荊州。郭嘉諫曰：“袁紹未除，呂蒙已拜于冀州，霸鎮、陸皇后為征北，遣使李肅守許都如何？”汜曰：“劉備、孫、吳，此又是司馬懿分付自中，而去取夏侯楙，吳將自率軍來救。懿即令人去請問：“我好此時有書，欲盡是我，是為何主骨肉之計矣！”孔明曰：“吾主無所用，今又何留我情？”孔明曰：“吾師女相待，奈朝廷有國，只在襄陽救劉備書。未知此事。又不見臣，先得其党有降。”布大怒，令許褚、張遼為前部，赶至江北，分調關而行。孟獲跪進。操欲思免死，心中愈加。施